In [44]:
from web3 import Web3
import json
import os

# Conectar a la red de Amoy de Polygon usando Web3
infura_url = 'https://polygon-amoy.infura.io/v3/50e449f7741f4ff7bc314929648a10ce'  # Reemplaza con tu Infura URL
web3 = Web3(Web3.HTTPProvider(infura_url))

In [33]:


# Verificar si estás conectado a la red
if web3.eth.chain_id is not None:
    print("Conectado a la red Amoy de Polygon")
else:
    raise Exception("No se pudo conectar a la red Amoy")

# Direcciones de los contratos
gestion_usuarios_address = Web3.to_checksum_address('0x4558FD325d2F03832e0F233857000cD76E442111')
contrato_permiso_address = Web3.to_checksum_address('0xc8d4a27811733e294B99f567910983C355d520DB')

# Cargar ABI de los contratos
with open("./Plataforma-Permisos/artifacts/contracts/GestionUsuarios.sol/GestionUsuarios.json", "r") as file1:
    gestion_usuarios_abi = json.load(file1)["abi"]

with open("./Plataforma-Permisos/artifacts/contracts/PermisoTrabajo.sol/PermisoTrabajo.json", "r") as file2:
    permiso_trabajo_abi = json.load(file2)["abi"]

# Inicializar contratos
gestion_usuarios_contract = web3.eth.contract(address=gestion_usuarios_address, abi=gestion_usuarios_abi)
permiso_trabajo_contract = web3.eth.contract(address=contrato_permiso_address, abi=permiso_trabajo_abi)

# Dirección del owner
private_key = '0xb98980ce3fa92e0b594214ea20108baa96a010d1f8c43a6a28e491a70815d86b'
direccion_owner = web3.eth.account.from_key(private_key)

#direccion_owner = web3.eth.accounts[0]

# Asignar un aprobador (solo el owner puede hacerlo)
def asignar_aprobador(direccion_aprobador):
    tx_hash = gestion_usuarios_contract.functions.asignarAprobador(direccion_aprobador).transact({
        'from': direccion_owner
    })
    receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Aprobador asignado. Transacción: {tx_hash.hex()}")

# Registrar un trabajador (solo el owner puede hacerlo)
def registrar_trabajador(direccion_trabajador):
    tx_hash = gestion_usuarios_contract.functions.registrarTrabajador(direccion_trabajador).transact({
        'from': direccion_owner
    })
    receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Trabajador registrado. Transacción: {tx_hash.hex()}")

# Solicitar un permiso (como trabajador)
def solicitar_permiso(direccion_solicitante,direccion_aprobador, fecha_inicio, fecha_fin, tipo):
    tx_hash = gestion_usuarios_contract.functions.solicitarPermiso(
        direccion_aprobador, fecha_inicio, fecha_fin, tipo
    ).transact({'from': direccion_solicitante})
    receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Permiso solicitado. Transacción: {tx_hash.hex()}")

# Aprobador aprueba un permiso
def aprobar_permiso(direccion_aprobador, permiso_id):
    tx_hash = gestion_usuarios_contract.functions.aprobarPermiso(
        contrato_permiso_address, permiso_id
    ).transact({'from': direccion_aprobador})
    receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Permiso aprobado. Transacción: {tx_hash.hex()}")

# Ejemplo de uso
# asignar_aprobador('0x...')
# registrar_trabajador('0x...')
# solicitar_permiso('0x...', '2023-09-01', '2023-09-05', 'vacaciones')
# aprobar_permiso('0x...', 1)


Conectado a la red Amoy de Polygon


In [38]:
direccion_aprobador = Web3.to_checksum_address("0xb7c02b30A2211c88c02A33586CA3B0AF686272C9")
direccion_solicitante = "0x4885B8939ad6c84772D537a01DB0fBc502076d24"

clave_aprobador = "0x2065944d38fb11999ccb99d6ebd815049e8ab3ecf029ac0b9443dcff5da7120e"
clave_solicitante = web3.eth.account.from_key("0xef54b4605f80d08cd8ffbf45749e89c97a55f3a3dcc2cb536613a5dbcb892cca")


fecha_inicio = "21/09/2024"
fecha_fin = "21/09/2024"
tipo = "Cita medica"
solicitar_permiso(clave_solicitante.address,direccion_aprobador, fecha_inicio, fecha_fin, tipo)

ExtraDataLengthError: The field extraData is 101 bytes, but should be 32. It is quite likely that you are connected to a POA chain. Refer to http://web3py.readthedocs.io/en/stable/middleware.html#proof-of-authority for more details. The full extraData is: HexBytes('0xd78301040183626f7288676f312e32322e37856c696e75780000000000000000c380c1c07377e6e5370be4e6d2ba0591e7174789b5161c3aec75c5e53a83db7e9ad8e4c94765721d6516e0327cdac1b1a123d6ffbf5f02c420ee04905adc7ba46b733a7d00')

In [52]:
from web3 import Web3
from web3.middleware import ExtraDataToPOAMiddleware
import json

# Dirección del owner
private_key = '0xb98980ce3fa92e0b594214ea20108baa96a010d1f8c43a6a28e491a70815d86b'
direccion_owner = web3.eth.account.from_key(private_key)

 # Reemplaza con tu URL de Infura
web3 = Web3(Web3.HTTPProvider(infura_url))

# Agregar el middleware para la red POA
web3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

# Verificar conexión
if web3.eth.chain_id is not None:
    print("Conectado a la red Amoy de Polygon")
else:
    raise Exception("No se pudo conectar a la red Amoy")

# Direcciones de los contratos
gestion_usuarios_address = web3.to_checksum_address('0x4558FD325d2F03832e0F233857000cD76E442111')
contrato_permiso_address = web3.to_checksum_address('0xc8d4a27811733e294B99f567910983C355d520DB')

# Cargar ABI de los contratos
with open("./Plataforma-Permisos/artifacts/contracts/GestionUsuarios.sol/GestionUsuarios.json", "r") as file1:
    gestion_usuarios_abi = json.load(file1)["abi"]

# Inicializar el contrato
gestion_usuarios_contract = web3.eth.contract(address=gestion_usuarios_address, abi=gestion_usuarios_abi)

# Datos para la solicitud
direccion_aprobador = web3.to_checksum_address("0xb7c02b30A2211c88c02A33586CA3B0AF686272C9")
clave_solicitante = "0x2065944d38fb11999ccb99d6ebd815049e8ab3ecf029ac0b9443dcff5da7120e"  # Clave privada del solicitante

fecha_inicio = "2024-09-21"
fecha_fin = "2024-09-21"
tipo = "Cita medica"

# Crear cuenta a partir de la clave privada
cuenta_solicitante = web3.eth.account.from_key(clave_solicitante)

def registrar_trabajador(direccion_trabajador):
    tx_hash = gestion_usuarios_contract.functions.registrarTrabajador(direccion_trabajador).transact({
        'from': direccion_owner
    })
    receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Trabajador registrado. Transacción: {tx_hash.hex()}")

# Solicitar permiso
def solicitar_permiso(direccion_aprobador, fecha_inicio, fecha_fin, tipo):
    tx_hash = gestion_usuarios_contract.functions.solicitarPermiso(
        contrato_permiso_address, fecha_inicio, fecha_fin, tipo
    ).transact({
        'from': cuenta_solicitante.address
    })
    receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Permiso solicitado. Transacción: {tx_hash.hex()}")




Conectado a la red Amoy de Polygon


In [53]:
direccion_trabajador = web3.to_checksum_address("0x4885B8939ad6c84772D537a01DB0fBc502076d24")
registrar_trabajador(cuenta_solicitante)

MismatchedABI: 
Could not identify the intended function with name `registrarTrabajador`, positional arguments with type(s) `(LocalAccount)` and keyword arguments with type(s) `{}`.
Found 1 function(s) with the name `registrarTrabajador`: ['registrarTrabajador(address)']
Function invocation failed due to no matching argument types.

In [54]:
# Llamar a la función
solicitar_permiso(direccion_aprobador, fecha_inicio, fecha_fin, tipo)

ContractLogicError: ('execution reverted: Solo un trabajador puede realizar esta accion', '0x08c379a00000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002d536f6c6f20756e2074726162616a61646f72207075656465207265616c697a6172206573746120616363696f6e00000000000000000000000000000000000000')